In [1]:
%load_ext autoreload
%autoreload 2
%config Completer.use_jedi=False

In [2]:
from os.path import join, expanduser, dirname
import pandas as pd
import sys
import os
import re
import warnings

warnings.filterwarnings(action='ignore')
home = expanduser('~')

src_path = '{}/zrp'.format(home)
sys.path.append(src_path)

from zrp.prepare.prepare import ZRP_Prepare
from zrp.prepare.utils import load_file

/Users/jay/opt/anaconda3/envs/zrp_test/lib/python3.9/site-packages/category_encoders/utils.py:5: DeprecationWarning: Please use `csr_matrix` from the `scipy.sparse` namespace, the `scipy.sparse.csr` namespace is deprecated.
  from scipy.sparse.csr import csr_matrix


In [3]:
nj_mayors  =  load_file("unit_test_data/2022-nj-mayors-sample.csv")

In [4]:
prepare = ZRP_Prepare()
## fit function just checkes the data quality 
prepare.fit(nj_mayors)

In [5]:
lookup_tables_config = load_json("unit_test_data/lookup_tables_config.json")
## loading inv_state_mapping file 
inv_state_map = load_json("unit_test_data/inv_state_mapping.json")

NameError: name 'load_json' is not defined

In [6]:
'/Users/jay/Documents/Zrp_unit_tests/zrp/zrp/prepare/../data/processed/geo/2019/Zest_Geo_Lookup_2019_State_34.parquet'

'/Users/jay/Documents/Zrp_unit_tests/zrp/zrp/prepare/../data/processed/geo/2019/Zest_Geo_Lookup_2019_State_34.parquet'

In [9]:
os.listdir('/Users/jay/Documents/Zrp_unit_tests/zrp/zrp/data')

['.DS_Store', 'processed']

In [36]:
curpath = '/Users/jay/Documents/zrp/zrp/prepare'
data_path = join(curpath, f'../data/processed')

In [37]:
geo_folder = os.path.join(data_path, "geo", lookup_tables_config['geo_year'])
acs_folder = os.path.join(data_path, 'acs', lookup_tables_config['acs_year'], lookup_tables_config['acs_span'])

In [38]:
gen_process = ProcessStrings(file_path=prepare.file_path)

In [39]:
data = nj_mayors

In [40]:
gen_process.fit(data)
data = gen_process.transform(data)

   [Start] Validating input data
     Number of observations: 462
     Is key unique: True
       (Warning!!) middle_name is 68.3982683982684% missing
Directory already exists
   [Completed] Validating input data

   Formatting P1
   Formatting P2
   reduce whitespace


In [41]:
inv_state_map = load_json(join(data_path, "inv_state_mapping.json"))

In [42]:
data['zest_in_state_fips'] = data[gen_process.state].replace(inv_state_map)

In [43]:
data.head(2)

,first_name,middle_name,last_name,house_number,street_address,city,state,zip_code,house_number_LEFT,house_number_RIGHT,zest_in_state_fips
ZEST_KEY,,,,,,,,,,,
2,GABE,None,PLUMER,782,FRENCHTOWN ROAD,MILFORD,NJ,08848,,782,34
4,ARI,None,BERNSTEIN,500,WEST CRESCENT AVENUE,ALLENDALE,NJ,07401,,500,34


In [44]:
# from zre.geo_geocoder import *
geocode = ZGeo(file_path=gen_process.file_path)

In [45]:
geo_grps = data.groupby([gen_process.state])

In [46]:
geo_dict = {}
for s, g in geo_grps:
    geo_dict[s] = g
gdkys = list(geo_dict.keys())
print("  The following states are included in the data:", gdkys)

  The following states are included in the data: ['NJ']


In [47]:
 if not set(gdkys) <= set(list(inv_state_map.keys())):
            raise ValueError("Provided unrecognizable state codes. Please use standard 2-letter abbreviation to indicate states to geocode, ex:'CA' for Californina")


In [48]:
geo_out = [] 
for s in tqdm(gdkys):
    print("   ... on state:", str(s))
    geo = inv_state_map[s].zfill(2)
    output = geocode.transform(geo_dict[s], geo, processed = True, replicate = True, save_table = True)
    geocode_out.append(output)
    break

  0%|                                                   | 0/462 [00:00<?, ?it/s][Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:    0.0s
[Parallel(n_jobs=-1)]: Done 184 tasks      | elapsed:    0.0s
[Parallel(n_jobs=-1)]: Done 434 tasks      | elapsed:    0.0s
100%|██████████████████████████████████████| 462/462 [00:00<00:00, 22226.70it/s]
[Parallel(n_jobs=-1)]: Done 462 out of 462 | elapsed:    0.0s finished


   ... on state: NJ

   Data is loaded
   [Start] Processing geo data
      ...address cleaning
      ...replicating address
         ...Base
         ...Number processing...
         House number dataframe expansion is complete! (n=462)
         ...Base
         ...Map street suffixes...
         ...Mapped & split by street suffixes...
         ...Number processing...

         Address dataframe expansion is complete! (n=900)
      ...formatting
   [Completed] Processing geo data
   [Start] Mapping geo data
      ...merge user input & lookup table
      ...mapping


  0%|                                                     | 0/1 [00:03<?, ?it/s]

   [Completed] Validating input geo data
Directory already exists
...Output saved
   [Completed] Mapping geo data


NameError: name 'geocode_out' is not defined